In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LayerNormalization,
    MultiHeadAttention, GlobalAveragePooling1D
)
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
file_path = r"F:\Xai_traderx\experiments\reldemo.csv"
df = pd.read_csv(file_path)

In [3]:
selected_features = [
    # Price Action
    'Close', 'High', 'Low', 'Open',
    
    # Short-Term Trends
    'SMA_7', 'SMA_10', 'EMA_10',
    
    # Volatility & Volume
    'bb_upper', 'bb_lower', 'OBV', 'ATR',
    
    # Optional (Test Impact)
    'EMA_50', 'RSI_14', 'MACD_Hist'
]

In [4]:
df_selected = df[selected_features]

In [5]:
# 1. Separate features and target
features = df_selected
target = df[['next_close']]

# 2. Scale features and target separately
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(target)

# 3. Train-test split (no shuffle)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, shuffle=False, test_size=0.2)

# 4. Define lookback
lookback = 4

# 5. Sequence creation function
def create_sequence(X, y, lookback):
    Xs, ys = [], []
    for i in range(len(X) - lookback):
        Xs.append(X[i:(i + lookback)])
        ys.append(y[i + lookback])
    return np.array(Xs), np.array(ys)

# 6. Create sequences
X_train_seq, y_train_seq = create_sequence(X_train, y_train, lookback)
X_test_seq, y_test_seq = create_sequence(X_test, y_test, lookback)

print("Shape of X_train_seq:", X_train_seq.shape)
print("Shape of y_train_seq:", y_train_seq.shape)
print("Shape of X_test_seq:", X_test_seq.shape)
print("Shape of y_test_seq:", y_test_seq.shape)

Shape of X_train_seq: (577, 4, 14)
Shape of y_train_seq: (577, 1)
Shape of X_test_seq: (142, 4, 14)
Shape of y_test_seq: (142, 1)


In [6]:
def transformer_model(input_shape, num_heads=4, dff=128, num_layers=2):
    inputs = Input(shape=input_shape)
    
    x = inputs
    for _ in range(num_layers):
        # Multi-Head Attention layer
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=64)(x, x)
        x = LayerNormalization()(x + attn_output)  
        
        # Feed-forward layer
        ff_output = Dense(dff, activation='relu')(x)
        ff_output = Dense(x.shape[-1])(ff_output)  # Project back to the original input shape
        x = LayerNormalization()(x + ff_output)  # Add & Normalize

    # Global Average Pooling
    x = GlobalAveragePooling1D()(x)
    
    # Fully connected output layer
    x = Dropout(0.2)(x)
    output = Dense(1)(x)

    # Define the model
    model = Model(inputs=inputs, outputs=output)
    return model

# Create the Transformer model
model = transformer_model(input_shape=(lookback, X_train_seq.shape[2]))

# Compile the model
optimizer = Nadam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss=Huber(), metrics=['mae'])

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_mae', patience=50, restore_best_weights=True),
    ModelCheckpoint('best_model_transformer.keras', save_best_only=True),
    ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=25, min_lr=1e-6)
]

# Fit the model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_test_seq, y_test_seq),
    epochs=500,
    batch_size=1,
    callbacks=callbacks,
    shuffle=False,
    verbose=1
)

print("Training Completed..")

Epoch 1/500
577/577 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.2354 - mae: 0.5551 - val_loss: 0.0261 - val_mae: 0.1969 - learning_rate: 1.0000e-04
Epoch 2/500
577/577 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0573 - mae: 0.2526 - val_loss: 0.0191 - val_mae: 0.1700 - learning_rate: 1.0000e-04
Epoch 3/500
577/577 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0378 - mae: 0.2112 - val_loss: 0.0285 - val_mae: 0.1952 - learning_rate: 1.0000e-04
Epoch 4/500
577/577 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0244 - mae: 0.1707 - val_loss: 0.0152 - val_mae: 0.1547 - learning_rate: 1.0000e-04
Epoch 5/500
577/577 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0163 - mae: 0.1392 - val_loss: 0.0149 - val_mae: 0.1514 - learning_rate: 1.0000e-04
Epoch 6/500
577/577 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.0140 - mae: 0.1325 - val_loss: 0.0176 - val_mae: 0.1607 - learning_rate: 1.0000e-04
Epoch 7/500
577/577 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0118 - mae: 0.1187 - val_loss: 0.0131 - val_mae: 0.1418

In [7]:
test_loss, test_mae = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0025 - mae: 0.0577 
Test Loss: 0.0028294844087213278
Test MAE: 0.06017225608229637


In [8]:
y_pred_scaled = model.predict(X_test_seq)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


In [9]:
# Reshape if needed
y_pred_scaled = y_pred_scaled.reshape(-1, 1)
y_test_seq = y_test_seq.reshape(-1, 1)

# Inverse transform
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test_seq)

In [12]:
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)
percentage_error = (mae / np.mean(y_true)) * 100
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2 Score: {r2:.4f}")
print(f"Percentage Error: {percentage_error:.2f}%")

MAE: 41.7824
RMSE: 52.2354
R2 Score: 0.8160
Percentage Error: 3.08%
